In [10]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.datasets import load_wine

# Load wine dataset
wine = load_wine()
X = wine.data


In [11]:
# Define clustering algorithms
kmeans = KMeans(random_state=42)
hierarchical = AgglomerativeClustering()
# Specify the number of components (clusters) for GMM
gmm = GaussianMixture(n_components=3, random_state=42)  # You can adjust the number of components as needed


In [12]:
# Define preprocessing approaches
preprocessing = {
    'No Preprocessing': lambda x: x,
    'Normalize': normalize,
    'Transform': lambda x: x,  # Replace with your own transformation function
    'PCA': lambda x: PCA(n_components=2).fit_transform(x),
    'T+N': lambda x: normalize(x),
    'T+N+PCA': lambda x: PCA(n_components=2).fit_transform(normalize(x))
}


In [13]:
# Create DataFrames to store the results for each algorithm
index = ['Silhouette', 'Calinski-Harabasz', 'Davies-Bouldin']
columns = pd.MultiIndex.from_product([['No Preprocessing', 'Normalize', 'Transform', 'PCA', 'T+N', 'T+N+PCA'], ['c=3', 'c=4', 'c=5']], names=['Preprocessing', 'Clusters'])
kmeans_df = pd.DataFrame(index=index, columns=columns)
hierarchical_df = pd.DataFrame(index=index, columns=columns)
gaussian_df = pd.DataFrame(index=index, columns=columns)

# Perform clustering for each combination of preprocessing and clusters
for preprocess_column, preprocess_func in preprocessing.items():
    for c in columns.get_level_values('Clusters').unique():
        X_preprocessed = preprocess_func(X)
        
        # Fit the clustering algorithms
        kmeans.fit(X_preprocessed)
        hierarchical.fit(X_preprocessed)
        gmm.fit(X_preprocessed)

        # Evaluate clustering quality
        kmeans_silhouette = silhouette_score(X_preprocessed, kmeans.labels_)
        kmeans_calinski = calinski_harabasz_score(X_preprocessed, kmeans.labels_)
        kmeans_bouldin = davies_bouldin_score(X_preprocessed, kmeans.labels_)

        hierarchical_silhouette = silhouette_score(X_preprocessed, hierarchical.labels_)
        hierarchical_calinski = calinski_harabasz_score(X_preprocessed, hierarchical.labels_)
        hierarchical_bouldin = davies_bouldin_score(X_preprocessed, hierarchical.labels_)

        gmm_silhouette = silhouette_score(X_preprocessed, gmm.predict(X_preprocessed))
        gmm_calinski = calinski_harabasz_score(X_preprocessed, gmm.predict(X_preprocessed))
        gmm_bouldin = davies_bouldin_score(X_preprocessed, gmm.predict(X_preprocessed))

        # Update the results DataFrames
        kmeans_df.loc[:, (preprocess_column, c)] = [kmeans_silhouette, kmeans_calinski, kmeans_bouldin]
        hierarchical_df.loc[:, (preprocess_column, c)] = [hierarchical_silhouette, hierarchical_calinski, hierarchical_bouldin]
        gaussian_df.loc[:, (preprocess_column, c)] = [gmm_silhouette, gmm_calinski, gmm_bouldin]


C:\Users\somya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\somya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\somya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set

In [14]:
# Save each DataFrame to a separate CSV file with technique name on top
kmeans_df.to_csv('kmeans_results.csv', index=True)
hierarchical_df.to_csv('hierarchical_results.csv', index=True)
gaussian_df.to_csv('gaussian_results.csv', index=True)

# Concatenate the CSV files with some space and technique names in between
output_file = 'compared_results.csv'
with open(output_file, 'w') as outfile:
    # Write K-Means technique name
    outfile.write('K-Means\n')
    with open('kmeans_results.csv', 'r') as infile:
        outfile.write(infile.read())
    outfile.write('\n\n')  # Add space between DataFrames
    
    # Write Hierarchical technique name
    outfile.write('Hierarchical\n')
    with open('hierarchical_results.csv', 'r') as infile:
        outfile.write(infile.read())
    outfile.write('\n\n')  # Add space between DataFrames
    
    # Write Gaussian technique name
    outfile.write('Gaussian\n')
    with open('gaussian_results.csv', 'r') as infile:
        outfile.write(infile.read())

print(f'Combined results with technique names saved to {output_file}')


Combined results with technique names saved to compared_results.csv
